In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.0MB 3.2MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 256kB 47.1MB/s 
     |████████████████████████████████| 163kB 41.8MB/s 


example2,   general optimization problem: 
\begin{align}
\min \ c^Tx\\
 Ax ≤ b\\
 l ≤ x ≤ u
\end{align}

---

In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
#creating a concrete model
model=ConcreteModel()

In [5]:
#defining no. of variables
N=20
# defining no. of constraints 
M=3

In [6]:
#defining coefficient(first 10 variables corresponds to stocks available in the company ,
# other 10 variables for those which need to be purchased from market  
# note: i and i+10 corresponds same product.
obj_coef_c=np.array([35,50,58,60,44,39,45,55,35,40,72,95,110,125,88,74,95,115,60,84])

In [7]:
#defining constraints coefficient in matrix of 3*20
const_coef_A=np.array([[80,75,75,60,55,55,40,35,30,30,80,75,75,60,55,55,40,35,30,30],
                       [15,15,10,20,25,10,50,15,30,55,15,15,10,20,25,10,50,15,30,55],
                       [5,10,15,20,20,35,10,50,40,15,5,10,15,20,20,35,10,50,40,15]])
print(const_coef_A)

[[80 75 75 60 55 55 40 35 30 30 80 75 75 60 55 55 40 35 30 30]
 [15 15 10 20 25 10 50 15 30 55 15 15 10 20 25 10 50 15 30 55]
 [ 5 10 15 20 20 35 10 50 40 15  5 10 15 20 20 35 10 50 40 15]]


In [8]:
#defining RHS value in constraint equation
const_rhs_b =np.array([45*250,35*250,20*250])

In [9]:
lower_bound = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
upper_bound = np.array([10,7,8,15,8,15,12,10,0,9,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf ])

In [10]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2]


In [11]:
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [12]:
#declare the decision variables in the model
model.x = Var(col_indices)


In [13]:
# creating  a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()

In [14]:
#defining constraints using for loop
for i in row_indices:
    model.constraints.add(sum(const_coef_A[i,j]*model.x[j] for j in col_indices)==(const_rhs_b[i]))
  

In [15]:
#adding the upper bound and lower bounds for the variables
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])


In [16]:
#defining objective function
model.objective=Objective(expr=sum(obj_coef_c[j]*model.x[j] for j in col_indices),sense=minimize)

In [17]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

1 Var Declarations
    x : Size=20, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  10.0 : False :  True :  Reals
          1 :     0 :  None :   7.0 : False :  True :  Reals
          2 :     0 :  None :   8.0 : False :  True :  Reals
          3 :     0 :  None :  15.0 : False :  True :  Reals
          4 :     0 :  None :   8.0 : False :  True :  Reals
          5 :     0 :  None :  15.0 : False :  True :  Reals
          6 :     0 :  None :  12.0 : False :  True :  Reals
          7 :     0 :  None :  10.0 : False :  True :  Reals
          8 :     

In [18]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144556 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [19]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16238.44
  Upper bound: 16238.44
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 21
  Number of nonzeros: 61
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.010906457901000977
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [20]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for j in col_indices:
  print('x[',j,'] = ', model.x[j].value)

print('\nConstraints')
model.constraints.display()


Objective =  16238.44

Decision Variables
x[ 0 ] =  10.0
x[ 1 ] =  7.0
x[ 2 ] =  8.0
x[ 3 ] =  15.0
x[ 4 ] =  8.0
x[ 5 ] =  15.0
x[ 6 ] =  12.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  9.0
x[ 10 ] =  28.6
x[ 11 ] =  0.0
x[ 12 ] =  0.0
x[ 13 ] =  0.0
x[ 14 ] =  0.0
x[ 15 ] =  0.0
x[ 16 ] =  0.0
x[ 17 ] =  0.0
x[ 18 ] =  52.64
x[ 19 ] =  84.76

Constraints
constraints : Size=3
    Key : Lower   : Body    : Upper
      1 : 11250.0 : 11250.0 : 11250.0
      2 :  8750.0 :  8750.0 :  8750.0
      3 :  5000.0 :  5000.0 :  5000.0


3.[R]#optimum value of objective function is **16238.44**\
 At optimality condition values of variables are 
 x[ 0 ] =  10.0\
x[ 1 ] =  7.0\
x[ 2 ] =  8.0\
x[ 3 ] =  15.0\
x[ 4 ] =  8.0\
x[ 5 ] =  15.0\
x[ 6 ] =  12.0\
x[ 7 ] =  0.0\
x[ 8 ] =  0.0\
x[ 9 ] =  9.0\
x[ 10 ] =  28.6\
x[ 11 ] =  0.0\
x[ 12 ] =  0.0\
x[ 13 ] =  0.0\
x[ 14 ] =  0.0\
x[ 15 ] =  0.0\
x[ 16 ] =  0.0\
x[ 17 ] =  0.0\
x[ 18 ] =  52.64\
x[ 19 ] =  84.76

4.[R] \
Three alloy which is ranked above all according to their quantity purchased from market .\
A10=84.76 Ton \
A9=52.64 Ton \
A1=28.6 Ton \
only these alloys are purchased from market so there is no low ranked alloy.

5.[R]\
alloy A1,A2,A3,A4,A5,A6,A7 and A10 is completely utilised,
since these product are available in the company stock which cost very less as compared to when purchased from market.therefore utilised fully to save cost.


6.[R]
alloy A2,A3,A4,A5,A6,A7 need not be purchased from market.
since required quantity for optimum condition is available in the stock.